In [4]:
#dependencies
import os
import pandas as pd
import numpy as np

In [5]:
#load in data
file = os.path.join('purchase_data.json')
pur_df = pd.read_json(file)
pur_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [6]:
#show total number of players only
total_players = len(pur_df['SN'].unique())
total_players_df = pd.DataFrame([{'Total Players': total_players}])
total_players_df.set_index('Total Players', inplace = True)
total_players_df

""
Total Players
573


In [7]:
#purcahse analysis (total)

#number of unique items
no_multiples = pur_df.drop_duplicates(['Item ID'], keep = 'last')
total_unique = len(no_multiples)

#total number of purchases
total_pur = pur_df['Price'].count

#total revenue
total_rev = round(pur_df['Price'].sum(),2)

#average purchase price (total revenue/total purchases)
avg_price = round(total_rev/total_pur(), 2)


#create data frame
pur_analysis = pd.DataFrame([{
    
    "Number of Unique Items": total_unique,
    'Average Purchase Price': avg_price,
    'Total Purchases': total_pur,
    'Total Revenue': total_rev}])

#check data frame
print(pur_analysis)

   Average Purchase Price  Number of Unique Items  \
0                    2.93                     183   

                                     Total Purchases  Total Revenue  
0  <bound method Series.count of 0      3.37\n1  ...        2286.33  


In [8]:
#format purchase analysis (total) data
pur_analysis.style.format({'Average Purchase Price': '${:.2f}', 'Total Revenue': '${:,.2f}'})

,Average Purchase Price,Number of Unique Items,Total Purchases,Total Revenue
0,$2.93,183,"<bound method Series.count of 0 3.37 1 2.32 2 2.46 3 1.36 4 1.27 5 1.73 6 4.57 7 3.32 8 2.77 9 4.53 10 3.81 11 1.55 12 4.06 13 3.68 14 2.46 15 4.77 16 1.14 17 1.65 18 3.71 19 4.89 20 2.33 21 1.79 22 2.52 23 2.36 24 1.96 25 1.27 26 3.90 27 2.29 28 4.23 29 2.46 ... 750 1.28 751 1.88 752 2.37 753 2.28 754 1.36 755 1.88 756 1.20 757 2.52 758 1.21 759 2.20 760 4.06 761 1.24 762 4.39 763 4.32 764 3.89 765 1.49 766 2.23 767 1.21 768 3.56 769 3.74 770 3.27 771 1.03 772 2.07 773 2.46 774 1.14 775 3.62 776 1.36 777 4.15 778 1.72 779 3.61 Name: Price, Length: 780, dtype: float64>","$2,286.33"


In [9]:
#Gender Demographics

In [10]:
# Percentage and count of players 
no_dup_players = pur_df.drop_duplicates(['SN'], keep ='last')
gender_count = no_dup_players['Gender'].value_counts().reset_index()
gender_count['% of Players'] = gender_count['Gender']/total_players * 100
#rename columns
gender_count.rename(columns = {'index': 'Gender', 'Gender': '# of Players'}, inplace = True)
#fix index
gender_count.set_index(['Gender'], inplace = True)
gender_count.style.format({"% of Players": "{:.1f}%"})


,# of Players,% of Players
Gender,,
Male,465,81.2%
Female,100,17.5%
Other / Non-Disclosed,8,1.4%


In [11]:
#Purchasing Analysis (Gender)


In [12]:
#purchase count by gender
pur_count_gen = pd.DataFrame(pur_df.groupby('Gender')['Gender'].count())

In [13]:
#Total Purchase Value
total_pur_val_gen = pd.DataFrame(pur_df.groupby('Gender')['Price'].sum())

In [14]:
#merge purchase count and total value
pur_analysis_gen = pd.merge(pur_count_gen, total_pur_val_gen, left_index = True, right_index = True)


In [15]:
#rename columns
pur_analysis_gen.rename(columns = {'Gender': '# of Purchases', 'Price':'Total Purchase Value'}, inplace=True)


In [16]:
#new column - average purchase price
pur_analysis_gen['Average Purchase Price'] = pur_analysis_gen['Total Purchase Value']/pur_analysis_gen['# of Purchases']


In [17]:
#add gender count
pur_analysis_gen = pur_analysis_gen.merge(gender_count, left_index = True, right_index = True)


In [18]:
#normalized totals (total purchase value/unique # of players by gender)
pur_analysis_gen['Normalized Totals'] = pur_analysis_gen['Total Purchase Value']/pur_analysis_gen['# of Players']
pur_analysis_gen

,# of Purchases,Total Purchase Value,Average Purchase Price,# of Players,% of Players,Normalized Totals
Gender,,,,,,
Female,136,382.91,2.815515,100,17.452007,3.829100
Male,633,1867.68,2.950521,465,81.151832,4.016516
Other / Non-Disclosed,11,35.74,3.249091,8,1.396161,4.467500


In [19]:
#remove number/percent of players. we don't need those columns.
del pur_analysis_gen['% of Players']
del pur_analysis_gen['# of Players']
pur_analysis_gen

,# of Purchases,Total Purchase Value,Average Purchase Price,Normalized Totals
Gender,,,,
Female,136,382.91,2.815515,3.829100
Male,633,1867.68,2.950521,4.016516
Other / Non-Disclosed,11,35.74,3.249091,4.467500


In [20]:
#fix formatting
pur_analysis_gen.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}', 'Normalized Totals': '${:.2f}'})

,# of Purchases,Total Purchase Value,Average Purchase Price,Normalized Totals
Gender,,,,
Female,136,$382.91,$2.82,$3.83
Male,633,$1867.68,$2.95,$4.02
Other / Non-Disclosed,11,$35.74,$3.25,$4.47


In [21]:
#age demographics

In [22]:
#create rows based on age ranges
pur_df.loc[(pur_df['Age'] < 10), 'age_bin'] = "< 10"
pur_df.loc[(pur_df['Age'] >= 10) & (pur_df['Age'] <= 14), 'age_bin'] = "10 - 14"
pur_df.loc[(pur_df['Age'] >= 15) & (pur_df['Age'] <= 19), 'age_bin'] = "15 - 19"
pur_df.loc[(pur_df['Age'] >= 20) & (pur_df['Age'] <= 24), 'age_bin'] = "20 - 24"
pur_df.loc[(pur_df['Age'] >= 25) & (pur_df['Age'] <= 29), 'age_bin'] = "25 - 29"
pur_df.loc[(pur_df['Age'] >= 30) & (pur_df['Age'] <= 34), 'age_bin'] = "30 - 34"
pur_df.loc[(pur_df['Age'] >= 35) & (pur_df['Age'] <= 39), 'age_bin'] = "35 - 39"
pur_df.loc[(pur_df['Age'] >= 40), 'age_bin'] = "> 40"

In [25]:
#pur_data[['age_bin', 'Age']].count() to check

In [28]:
# count purchases by age bin - usingscreen names (non-unique)
pur_count_age = pd.DataFrame(pur_df.groupby('age_bin')['SN'].count())

In [29]:
#find averageg price of purchases by age bin
avg_price_age = pd.DataFrame(pur_df.groupby('age_bin')['Price'].mean())

In [30]:
#finds total purchase value by age bin
tot_pur_age = pd.DataFrame(pur_df.groupby('age_bin')['Price'].sum())

In [31]:
#delete duplicate screen names while only keeping last, then counts # of unique players by age bin
no_dup_age = pd.DataFrame(pur_df.drop_duplicates('SN', keep = 'last').groupby('age_bin')['SN'].count())

In [32]:
#merge
merge_age = pd.merge(pur_count_age, avg_price_age, left_index = True, right_index = True).merge(tot_pur_age, left_index = True, right_index = True).merge(no_dup_age, left_index = True, right_index = True)

In [33]:
#renames column
merge_age.rename(columns = {"SN_x": "# of Purchases", "Price_x": "Average Purchase Price", "Price_y": "Total Purchase Value", "SN_y": "# of Purchasers"}, inplace = True)


In [34]:
#calculates normalized totals
merge_age['Normalized Totals'] = merge_age['Total Purchase Value']/merge_age['# of Purchasers']

In [35]:
#fix index and format
merge_age.index.rename("Age", inplace = True)
merge_age.style.format({'Average Purchase Price': '${:.2f}', 'Total Purchase Value': '${:.2f}', 'Normalized Totals': '${:.2f}'})


,# of Purchases,Average Purchase Price,Total Purchase Value,# of Purchasers,Normalized Totals
Age,,,,,
10 - 14,35,$2.77,$96.95,23,$4.22
15 - 19,133,$2.91,$386.42,100,$3.86
20 - 24,336,$2.91,$978.77,259,$3.78
25 - 29,125,$2.96,$370.33,87,$4.26
30 - 34,64,$3.08,$197.25,47,$4.20
35 - 39,42,$2.84,$119.40,27,$4.42
< 10,28,$2.98,$83.46,19,$4.39
> 40,17,$3.16,$53.75,11,$4.89


In [36]:
#top spenders
#Identify the the top 5 spenders in the game by total purchase value, then list (in a table):

In [42]:
#Group by sn to find, total purchase per person, number of purchases per person, and average price price per person
purchase_amt_by_SN = pd.DataFrame(pur_df.groupby('SN')['Price'].sum())
num_purchase_by_SN = pd.DataFrame(pur_df.groupby('SN')['Price'].count())
avg_purchase_by_SN = pd.DataFrame(pur_df.groupby('SN')['Price'].mean())

#merge df's
merged_top5 = pd.merge(purchase_amt_by_SN, num_purchase_by_SN, left_index = True, right_index = True).merge(avg_purchase_by_SN, left_index=True, right_index=True)


In [43]:
# rename columns
merged_top5.rename(columns = {'Price_x': 'Total Purchase Value', 'Price_y':'Purchase Count', 'Price':'Average Purchase Price'}, inplace = True)


In [44]:
#sort highest to lowest purchase value
merged_top5.sort_values('Total Purchase Value', ascending = False, inplace=True)

In [45]:
#fix formatting
merged_top5 = merged_top5.head()
merged_top5.style.format({'Total Purchase Value': '${:.2f}', 'Average Purchase Price': '${:.2f}'})

,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Undirrala66,$17.06,5,$3.41
Saedue76,$13.56,4,$3.39
Mindimnya67,$12.74,4,$3.18
Haellysu29,$12.73,3,$4.24
Eoda93,$11.58,3,$3.86


In [46]:
#Most Popular Items
#Identify the 5 most popular items by purchase count, then list (in a table)

In [47]:
# get a count of each item by grouping by Item ID and number of occurances
top5_items_ID = pd.DataFrame(pur_df.groupby('Item ID')['Item ID'].count())


In [49]:
#sort from high to low total purchase count
top5_items_ID.sort_values('Item ID', ascending = False, inplace = True)

In [53]:
#keep the first 5 rows
top5_items_ID = top5_items_ID.iloc[0:5][:]
#top5_items_ID

In [55]:
#total purchase value of each item
top5_items_total = pd.DataFrame(pur_df.groupby('Item ID')['Price'].sum())
#top5_items_ID

In [88]:
#merge purcahse count with total purcahse value and drop dups
top5_items = pd.merge(top5_items_ID, top5_items_total, left_index = True, right_index = True)
no_dup_items = pur_df.drop_duplicates(['Item ID'], keep = 'last')
#top5_items

In [94]:
# merge to get all other info from the top 5 using the no dup df
top5_merge_ID = pd.merge(top5_items, no_dup_items, left_index = True, right_on = 'Item ID')
top5_items_ID = top5_merge_ID[['Item ID', 'Item Name', 'Item ID_x', 'Price_y', 'Price_x']]
top5_merge_ID.set_index(['Item ID'], inplace = True)
top5_merge_ID.rename(columns =  {'Item ID_x': 'Purchase Count', 'Price_y': 'Item Price', 'Price_x': 'Total Purchase Value'}, inplace=True)
top5_merge_ID.style.format({'Item Price': '${:.2f}', 'Total Purchase Value': '${:.2f}'})

,Purchase Count,Total Purchase Value,Age,Gender,Item ID_y,Item Name,Item Price,SN,age_bin
Item ID,,,,,,,,,
39,11,$25.85,26,Male,39,"Betrayal, Whisper of Grieving Widows",$2.35,Aeduera68,25 - 29
84,11,$24.53,22,Female,84,Arcane Gem,$2.23,Nitherian58,20 - 24
31,9,$18.63,15,Male,31,Trickster,$2.07,Sidap51,15 - 19
175,9,$11.16,28,Male,175,Woeful Adamantite Claymore,$1.24,Raeduerin33,25 - 29
13,9,$13.41,15,Male,13,Serenity,$1.49,Aerithnucal56,15 - 19


In [78]:
#Most Profitable Items - Identify the 5 most profitable items by total purchase value, then list (in a table)

In [100]:
top_5_profit = pd.DataFrame(pur_df.groupby('Item ID')['Price'].sum())
top_5_profit.sort_values('Price', ascending = False, inplace = True)
top_5_profit = top_5_profit.iloc[0:5][:]
pur_count_profit = pd.DataFrame(pur_df.groupby('Item ID')['Item ID'].count())

top_5_profit = pd.merge(top_5_profit, pur_count_profit, left_index = True, right_index = True, how = 'left')
top_5_merge_profit = pd.merge(top_5_profit, no_dup_items, left_index = True, right_on = 'Item ID', how = 'left')
top_5_merge_profit = top_5_merge_profit[['Item ID', 'Item Name', 'Item ID_x', 'Price_y','Price_x']]
top_5_merge_profit.set_index(['Item ID'], inplace=True)
top_5_merge_profit.rename(columns = {'Item ID_x': 'Purchase Count', 'Price_y': 'Item Price', 'Price_x': 'Total Purchase Value'}, inplace = True)
top_5_merge_profit.style.format({'Item Price': '${:.2f}', 'Total Purchase Value': '${:.2f}'})

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
